# Resturant Review Sentiment - Data Cleaning 
### Matthew Newton
* The goal of this notebook is to import and clean the dataset to be used in the Machine Learning models. 
* The data will be cleaned once and then saved to speed up the run times in the Machine Learning notebooks.

# Import data

In [2]:
import pandas as pd
import pickle

# Import as Pandas dataframe
df_review = pd.read_pickle("./data/reviews.pkl")
df_rest = pd.read_pickle("./data/restaurants.pkl")

# Examine datasets

In [3]:
df_review

,reviewId,restaurantId,title,text,date,rating
2209877,518970207,5815702,Excellent Lunch,My husband and I enjoyed an amazing lunch toda...,"August 29, 2017",5
1874426,503617969,4894321,Best Brunch Ever,"I LOVE this place, it's brunch was absolutely ...","July 19, 2017",5
1917033,306422966,1044542,A good light lunch with friends,I enjoyed inventive salads and wine with a goo...,"September 2, 2015",5
1664311,314883941,4470762,Simple food and good price,"Very basic selection of hamburgers, hot dogs a...","September 29, 2015",4
432828,650813333,14137170,Ceru - one of the best taste sensations!,This is my second visit and once again the spi...,"February 7, 2019",5
...,...,...,...,...,...,...
827653,446979385,7257981,great cafe on the Charing Cross Road,I often visit the Foyles cafe whether shopping...,"December 27, 2016",5
1771142,311624284,5234147,Rich and tasty lunch menu.,My first visit to the Merchant's Tavern was a ...,"September 18, 2015",4
1142841,462663906,1501386,Good food and service,We visited with friends on a London break. We ...,"February 25, 2017",4
1831734,603852832,14888129,"Fantastic food, horrid service","My friend booked last Saturday for 6 of us, wh...","August 7, 2018",3


In [4]:
df_rest

,id,name,priceInterval,rating,type
0,2429316,Assenheims 56,$,4.5,"Fast Food, South American"
1,11848599,Assenheims 56,$,4.0,"French, European"
2,12124190,Assenheims bar and grill,$$ - $$$,4.5,"Peruvian, Latin"
3,4023305,The Association Coffee,$$ - $$$,4.5,"Coffee & Tea, Cafe"
4,10036242,Association Coffee,$$ - $$$,4.0,"Coffee & Tea, Cafe"
...,...,...,...,...,...
19135,10497068,Brickwood Coffee & Bread,$$ - $$$,4.0,"Coffee & Tea, British"
19136,8685508,Brickwood Coffee & Bread,$$ - $$$,4.0,"Coffee & Tea, Cafe"
19137,1015992,The Bridge,$$ - $$$,4.5,International
19138,3681921,The Bridge,$$ - $$$,3.0,


In [5]:
# Check for missing values
print(df_review.isnull().sum())

reviewId        0
restaurantId    0
title           0
text            0
date            0
rating          0
dtype: int64


In [6]:
# Basic statistics of the ratings
print(df_review['rating'].describe())

count    2.044046e+06
mean     4.036257e+00
std      1.219886e+00
min      1.000000e+00
25%      4.000000e+00
50%      4.000000e+00
75%      5.000000e+00
max      5.000000e+00
Name: rating, dtype: float64


In [7]:
# Distribution of review lengths
df_review['review_length'] = df_review['text'].apply(len)
print(df_review['review_length'].describe())

count    2.044046e+06
mean     5.308633e+02
std      4.670396e+02
min      2.000000e+00
25%      2.310000e+02
50%      3.970000e+02
75%      6.690000e+02
max      2.212700e+04
Name: review_length, dtype: float64


# Data Cleaning and Processing
* The duplicates from the training and resturant datasets are removed (if there are any).
* The resturant data can be added as features into the training set using the resturant ID
* From the resturant data, the type and price interval can be used as features in the training set. The average rating of the resturant could be useful, however it could lead to overfitting from that feature.
* Our training set X_i, should contain the features: price, type, text, title, date:
    * Price should be converted to a value between 1 and 3 based on the number of '$'s.
    * Date should be converted to an integer value and then scaled/normalised using Gaussian when fitted.
    * Type, text and title should be cleaned so that they can be effectively converted to features using NLP techniques for linear regression/classification and decision tree models.
* The target output is the rating, which can only be descrete values from 1-5, multiclass classifcation can be used. It may be simplier to consider a linear ouput (continous) and then round to the nearest integer (1,2,3,4,5), however the results were not as promising.

# Remove duplicates

In [8]:
df_review = df_review.drop_duplicates()
df_rest = df_rest.drop_duplicates()

# Incorporate resturant data into training data
* Merge based on resturant ID in the training dataset and the ID in the resturant dataset.
* Remove unneeded columns from training data.

In [9]:
# Rename resturant rating to avoid overlapping column names
df_rest = df_rest.rename(columns={'id':'restaurantId', 'rating':'rest_rating'})
df_review = pd.merge(df_review, df_rest, on='restaurantId', how='left')

# Remove: reviewId, restaurantId, name, rest_rating
df_review = df_review.drop(columns=['reviewId', 'restaurantId', 'name', 'rest_rating'])
df_review

,title,text,date,rating,review_length,priceInterval,type
0,Excellent Lunch,My husband and I enjoyed an amazing lunch toda...,"August 29, 2017",5,1424,$$ - $$$,British
1,Best Brunch Ever,"I LOVE this place, it's brunch was absolutely ...","July 19, 2017",5,236,$$ - $$$,"Coffee & Tea, Cafe"
2,A good light lunch with friends,I enjoyed inventive salads and wine with a goo...,"September 2, 2015",5,409,$$ - $$$,"British, Pub"
3,Simple food and good price,"Very basic selection of hamburgers, hot dogs a...","September 29, 2015",4,616,$$ - $$$,"Quick Bites, American"
4,Ceru - one of the best taste sensations!,This is my second visit and once again the spi...,"February 7, 2019",5,559,$$ - $$$,"Mediterranean, Turkish"
...,...,...,...,...,...,...,...
2044041,great cafe on the Charing Cross Road,I often visit the Foyles cafe whether shopping...,"December 27, 2016",5,461,$$ - $$$,"Coffee & Tea, Cafe"
2044042,Rich and tasty lunch menu.,My first visit to the Merchant's Tavern was a ...,"September 18, 2015",4,666,$$ - $$$,"Bar, European"
2044043,Good food and service,We visited with friends on a London break. We ...,"February 25, 2017",4,179,$$$$,"Steakhouse, British"
2044044,"Fantastic food, horrid service","My friend booked last Saturday for 6 of us, wh...","August 7, 2018",3,1197,$,Cafe


# Convert price and date data into integer values
* Convert '$' in price into a value between 1-3.
* Convert date into an integer.
* These can be scaled later if used in regression/classification models.

In [10]:
# Find unique values in prices to create a dictionary
df_review['priceInterval'].unique() 
# output: array(['$$ - $$$', '$$$$', '$', '', nan], dtype=object)

# Fill in gaps with the median price (which was computed as 2)
df_review['priceInterval'] = df_review['priceInterval'].replace({'$$ - $$$' : 2, '$$$$' : 3, '$' : 1, '' : 2, 'nan' : 2})

# Convert date to an integer value
df_review['date'] = pd.to_datetime(df_review['date'], format='%B %d, %Y')
reference_date = pd.Timestamp('2010-01-01')
df_review['date'] = (df_review['date'] - reference_date).dt.days
df_review

/var/folders/32/936725p16d38l3l5gt6yy0z00000gp/T/ipykernel_12616/3649369530.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_review['priceInterval'] = df_review['priceInterval'].replace({'$$ - $$$' : 2, '$$$$' : 3, '$' : 1, '' : 2, 'nan' : 2})


,title,text,date,rating,review_length,priceInterval,type
0,Excellent Lunch,My husband and I enjoyed an amazing lunch toda...,2797,5,1424,2.0,British
1,Best Brunch Ever,"I LOVE this place, it's brunch was absolutely ...",2756,5,236,2.0,"Coffee & Tea, Cafe"
2,A good light lunch with friends,I enjoyed inventive salads and wine with a goo...,2070,5,409,2.0,"British, Pub"
3,Simple food and good price,"Very basic selection of hamburgers, hot dogs a...",2097,4,616,2.0,"Quick Bites, American"
4,Ceru - one of the best taste sensations!,This is my second visit and once again the spi...,3324,5,559,2.0,"Mediterranean, Turkish"
...,...,...,...,...,...,...,...
2044041,great cafe on the Charing Cross Road,I often visit the Foyles cafe whether shopping...,2552,5,461,2.0,"Coffee & Tea, Cafe"
2044042,Rich and tasty lunch menu.,My first visit to the Merchant's Tavern was a ...,2086,4,666,2.0,"Bar, European"
2044043,Good food and service,We visited with friends on a London break. We ...,2612,4,179,3.0,"Steakhouse, British"
2044044,"Fantastic food, horrid service","My friend booked last Saturday for 6 of us, wh...",3140,3,1197,1.0,Cafe


# Save datasets here before NLTK cleaning

In [11]:
#with open('./cleaned_data/reviews_cleaned.pickle', 'wb') as output:
#    pickle.dump(df_review, output)

# Use NLTK to process text, type and title features

In [12]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords list
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def process_text(text):
    if not isinstance(text, str): # Fixes error with inputs of type float
        text = ''
        
    # Remove punctuation and digits
    text = re.sub(f'[{string.punctuation}]', ' ', text)
    text = re.sub(r'\W', ' ', text) # Removes alternative characters
    text = re.sub(r'\s+', ' ', text) # Removes extra spaces
    text = re.sub(r'\d+', '', text)
    
    # Tokenize and remove stopwords
    words = word_tokenize(text.lower())
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    # Join words back into a single string
    return ' '.join(words)

col_names = ['text', 'title', 'type']

# Make all words lowercase and apply processing techniques to text
for col in col_names:
    df_review[col] = df_review[col].str.lower()
    df_review[col] = df_review[col].apply(process_text)   
    
df_review

[nltk_data] Downloading package punkt to /Users/mnewt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/mnewt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mnewt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,title,text,date,rating,review_length,priceInterval,type
0,excellent lunch,husband enjoyed amazing lunch today executed e...,2797,5,1424,2.0,british
1,best brunch ever,love place brunch absolutely scrumptious got r...,2756,5,236,2.0,coffee tea cafe
2,good light lunch friend,enjoyed inventive salad wine good friend easte...,2070,5,409,2.0,british pub
3,simple food good price,basic selection hamburger hot dog fry price ex...,2097,4,616,2.0,quick bite american
4,ceru one best taste sensation,second visit spice aroma dish mouthwateringly ...,3324,5,559,2.0,mediterranean turkish
...,...,...,...,...,...,...,...
2044041,great cafe charing cross road,often visit foyles cafe whether shopping foyle...,2552,5,461,2.0,coffee tea cafe
2044042,rich tasty lunch menu,first visit merchant tavern good one stylish d...,2086,4,666,2.0,bar european
2044043,good food service,visited friend london break enjoyed food good ...,2612,4,179,3.0,steakhouse british
2044044,fantastic food horrid service,friend booked last saturday u arrived told tak...,3140,3,1197,1.0,cafe


# Save dataset
Save as a pickle file to be used in other notebooks.

In [13]:
#with open('./cleaned_data/reviews_cleaned_nltk.pickle', 'wb') as output:
#    pickle.dump(df_review, output)